# Working with key/value pair RDDs

[Introduction to Spark with Python, by Jose A. Dianes](https://github.com/jadianes/spark-py-notebooks)

Spark provides specific functions to deal with RDDs which elements are key/value pairs. They are usually used to perform aggregations and other processings by key.  

In this notebook we will show how, by working with key/value pairs, we can process our network interactions dataset in a more practical and powerful way than that used in previous notebooks. Key/value pair aggregations will show to be particularly effective when trying to explore each type of tag in our network attacks, in an individual way.  

## Getting the data and creating the RDD

As we did in our first notebook, we will use the reduced dataset (10 percent) provided for the [KDD Cup 1999](http://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html), containing nearly half million network interactions. The file is provided as a Gzip file that we will download locally.  

In [1]:
import urllib
from urllib.request import urlretrieve
f = urlretrieve ("http://kdd.ics.uci.edu/databases/kddcup99/kddcup.data_10_percent.gz", "kddcup.data_10_percent.gz")

In [2]:
data_file = "./kddcup.data_10_percent.gz"
raw_data = sc.textFile(data_file)

## Creating a pair RDD for interaction types

In this notebook we want to do some exploratory data analysis on our network interactions dataset. More concretely we want to profile each network interaction type in terms of some of its variables such as duration. In order to do so, we first need to create the RDD suitable for that, where each interaction is parsed as a CSV row representing the value, and is put together with its corresponding tag as a key.  

Normally we create key/value pair RDDs by applying a function using `map` to the original data. This function returns the corresponding pair for a given RDD element. We can proceed as follows.  

In [3]:
csv_data = raw_data.map(lambda x: x.split(","))
key_value_data = csv_data.map(lambda x: (x[41], x)) # x[41] contains the network interaction tag

We have now our key/value pair data ready to be used. Let's get the first element in order to see how it looks like.  

In [4]:
key_value_data.take(1)

[('normal.',
  ['0',
   'tcp',
   'http',
   'SF',
   '181',
   '5450',
   '0',
   '0',
   '0',
   '0',
   '0',
   '1',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '0',
   '8',
   '8',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '1.00',
   '0.00',
   '0.00',
   '9',
   '9',
   '1.00',
   '0.00',
   '0.11',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   '0.00',
   'normal.'])]

## Data aggregations with key/value pair RDDs

We can use all the transformations and actions available for normal RDDs with key/value pair RDDs. We just need to make the functions work with pair elements. Additionally, Spark provides specific functions to work with RDDs containing pair elements. They are very similar to those available for general RDDs.  

For example, we have a `reduceByKey` transformation that we can use as follows to calculate the total duration of each network interaction type.  

In [5]:
key_value_duration = csv_data.map(lambda x: (x[41], float(x[0]))) 
durations_by_key = key_value_duration.reduceByKey(lambda x, y: x + y)

durations_by_key.collect()

[('portsweep.', 1991911.0),
 ('neptune.', 0.0),
 ('satan.', 64.0),
 ('pod.', 0.0),
 ('multihop.', 1288.0),
 ('back.', 284.0),
 ('warezclient.', 627563.0),
 ('nmap.', 0.0),
 ('smurf.', 0.0),
 ('guess_passwd.', 144.0),
 ('ftp_write.', 259.0),
 ('imap.', 72.0),
 ('land.', 0.0),
 ('loadmodule.', 326.0),
 ('buffer_overflow.', 2751.0),
 ('perl.', 124.0),
 ('ipsweep.', 43.0),
 ('rootkit.', 1008.0),
 ('phf.', 18.0),
 ('teardrop.', 0.0),
 ('warezmaster.', 301.0),
 ('normal.', 21075991.0),
 ('spy.', 636.0)]

We have a specific counting action for key/value pairs.  

In [7]:
counts_by_key = key_value_data.countByKey()
counts_by_key

defaultdict(int,
            {'back.': 2203,
             'buffer_overflow.': 30,
             'ftp_write.': 8,
             'guess_passwd.': 53,
             'imap.': 12,
             'ipsweep.': 1247,
             'land.': 21,
             'loadmodule.': 9,
             'multihop.': 7,
             'neptune.': 107201,
             'nmap.': 231,
             'normal.': 97278,
             'perl.': 3,
             'phf.': 4,
             'pod.': 264,
             'portsweep.': 1040,
             'rootkit.': 10,
             'satan.': 1589,
             'smurf.': 280790,
             'spy.': 2,
             'teardrop.': 979,
             'warezclient.': 1020,
             'warezmaster.': 20})

### Using `combineByKey`

This is the most general of the per-key aggregation functions. Most of the other per-key combiners are implemented using it. We can think about it as the `aggregate` equivalent since it allows the user to return values that are not the same type as our input data.

For example, we can use it to calculate per-type average durations as follows.  

In [8]:
sum_counts = key_value_duration.combineByKey(
    (lambda x: (x, 1)), # the initial value, with value x and count 1
    (lambda acc, value: (acc[0]+value, acc[1]+1)), # how to combine a pair value with the accumulator: sum value, and increment count
    (lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])) # combine accumulators
)

sum_counts.collectAsMap()

{'back.': (284.0, 2203),
 'buffer_overflow.': (2751.0, 30),
 'ftp_write.': (259.0, 8),
 'guess_passwd.': (144.0, 53),
 'imap.': (72.0, 12),
 'ipsweep.': (43.0, 1247),
 'land.': (0.0, 21),
 'loadmodule.': (326.0, 9),
 'multihop.': (1288.0, 7),
 'neptune.': (0.0, 107201),
 'nmap.': (0.0, 231),
 'normal.': (21075991.0, 97278),
 'perl.': (124.0, 3),
 'phf.': (18.0, 4),
 'pod.': (0.0, 264),
 'portsweep.': (1991911.0, 1040),
 'rootkit.': (1008.0, 10),
 'satan.': (64.0, 1589),
 'smurf.': (0.0, 280790),
 'spy.': (636.0, 2),
 'teardrop.': (0.0, 979),
 'warezclient.': (627563.0, 1020),
 'warezmaster.': (301.0, 20)}

We can see that the arguments are pretty similar to those passed to `aggregate` in the previous notebook. The result associated to each type is in the form of a pair. If we want to actually get the averages, we need to do the division before collecting the results.  

In [16]:
sum_counts.take(1)

[('portsweep.', (1991911.0, 1040))]

In [15]:
#Dunno why this didn't work, maybe map() doesn't know how to deal with what it's given?
#duration_means_by_type = sum_counts.map(lambda (key,value): (key, round(value[0]/value[1],3))).collectAsMap()
duration_means_by_type = sum_counts.map(lambda key,value: key, round(value[0]/value[1],3) ).collectAsMap()

# Print them sorted
for tag in sorted(duration_means_by_type, key=duration_means_by_type.get, reverse=True):
    print(tag, duration_means_by_type[tag])

AttributeError: 'dict' object has no attribute 'map'

A small step into understanding what makes a network interaction be considered an attack.